<h3>Mini-Project</h3>
<p>Develop a movie recommendation model using the scikit-learn library in python. Refer <a href="https://github.com/rashida048/Some-NLP-Projects/blob/master/movie_dataset.csv">dataset</a>. </p>
<hr>
<p><b>Group members:</b></p>
<p><b>31108</b> - Medha Badamikar</p>
<p><b>31117</b> - Udayan Chavan</p>
<hr>

Import required dependencies.

In [41]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

Import movie_dataset.csv as a dataframe.

In [42]:
data = pd.read_csv("movie_dataset.csv")
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


Get a description of the dataset.

In [58]:
data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


Print all available columns.

In [43]:
print(data.columns.tolist())

['index', 'budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew', 'director']


Cleaning up null values and combining all the important features for analysis.

In [44]:
features = ['keywords', 'cast', 'genres', 'director', 'tagline']

for feature in features:
    data[feature] = data[feature].fillna('')
    
def combine_features(row):
    try:
        return row['keywords'] + " " + row['cast'] + " " + row['genres'] + " " + row['director'] + " " + row['tagline']
    except:
        print("Error:", row)
        
data["combined_features"] = data.apply(combine_features, axis=1)

A function to find movie title from its index.

In [45]:
def title_from_index(index):
    return data[data.index == index]["title"].values[0]

A function to find movie index from it's title. Using get_close_matches function from the difflib library, we can return index of the closest matching title.

In [46]:
def index_from_title(title):
    title_list = data['title'].tolist()
    common = difflib.get_close_matches(title, title_list, 1)
    titlesim = common[0]
    return data[data.title == titlesim]["index"].values[0]

<p> Using CountVectorizer and fit_transform on combined_features to fit it accordingly, we calculate the cosine similarity of the count matrix. </p>

<p>1. Get movie index from the user input by using index_from_title function.</p>
<p>2. Use this movie index to pass it to cosine sum and enumerate the output as a list.</p>
<p>3. Sort this list in reverse order and get the number of movies to be recommended from the user.</p>
<p>4. Print user-defined number of recommended movies.</p>


In [61]:
cv  = CountVectorizer()

count_matrix = cv.fit_transform(data["combined_features"])
cosine_sim = cosine_similarity(count_matrix)

user_movie = input("Enter a movie: ")
movie_index = index_from_title(user_movie)

similar_movies = list(enumerate(cosine_sim[movie_index]))
similar_movies_sorted = sorted(similar_movies,key=lambda x:x[1], reverse=True)

i = 0

n_rec_movies = int(input("Enter number of movies to be recommended: "))

print("\nRecommended movies: \n")

for rec_movie in similar_movies_sorted:
    if(i!=0):
        print(i, ". ", title_from_index(rec_movie[0]), sep="")
        
    i=i+1
    if i > n_rec_movies:
        break

Enter a movie: The devil all the time
Enter number of movies to be recommended: 10

Recommended movies: 

1. They Will Have to Kill Us First
2. Wild Grass
3. One to Another
4. Mozart's Sister
5. Elite Squad
6. Evil Words
7. Underdogs
8. Le Havre
9. The Second Mother
10. Mississippi Mermaid


<hr><hr>
<h3> Sample Output: </h3>

<p><b>Enter a movie:</b> The Great Gatsby</p>
<p><b>Enter number of movies to be recommended:</b> 10</p>

<p>Recommended movies:</p> 

<p>1. The Scarlet Letter</p>
<p>2. Stand by Me</p>
<p>3. What to Expect When You're Expecting</p>
<p>4. The Number 23</p>
<p>5. Forrest Gump</p>
<p>6. The Ice Storm</p>
<p>7. The Dead Zone</p>
<p>8. Indignation</p>
<p>9. The Spectacular Now</p>
<p>10. Shutter Island</p>

<hr>


<p><b>Enter a movie:</b> The Dark Knight</p>
<p><b>Enter number of movies to be recommended:</b> 25</p>

<p>Recommended movies:</p>

<p>1. The Dark Knight Rises</p>
<p>2. Batman Begins</p>
<p>3. Kick-Ass</p>
<p>4. The Prestige</p>
<p>5. Kick-Ass 2</p>
<p>6. The Killer Inside Me</p>
<p>7. Batman & Robin</p>
<p>8. Batman Returns</p>
<p>9. In Too Deep</p>
<p>10. Batman</p>
<p>11. Harry Brown</p>
<p>12. Amidst the Devil's Wings</p>
<p>13. Kill the Messenger</p>
<p>14. Hitman</p>
<p>15. Defendor</p>
<p>16. Harsh Times</p>
<p>17. 3 Days to Kill</p>
<p>18. Mi America</p>
<p>19. The Pledge</p>
<p>20. Half Past Dead</p>
<p>21. Insomnia</p>
<p>22. The Raid</p>
<p>23. Takers</p>
<p>24. Blood Ties</p>
<p>25. Pain & Gain</p>

<hr>

<p><b>Enter a movie:</b> Never Back Down</p>
<p><b>Enter number of movies to be recommended:</b> 5</p>

<p>Recommended movies:</p>

<p>1. Warrior</p>
<p>2. Kickboxer: Vengeance</p>
<p>3. Street Fighter: The Legend of Chun-Li</p>
<p>4. Curse of the Golden Flower</p>
<p>5. Reign of Assassins</p>

<hr>

<p><b>Enter a movie:</b> Fury</p>
<p><b>Enter number of movies to be recommended:</b> 10</p>

<p>Recommended movies:</p>

<p>1. Saving Private Ryan</p>
<p>2. Unbroken</p>
<p>3. Hart's War</p>
<p>4. City of Life and Death</p>
<p>5. The Monuments Men</p>
<p>6. Little Boy</p>
<p>7. Saints and Soldiers</p>
<p>8. A Bridge Too Far</p>
<p>9. The Thin Red Line</p>
<p>10. Miracle at St. Anna</p>

<hr>